In [ ]:
import tensorpac
import mne
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorpac import EventRelatedPac
from tensorpac.signals import pac_signals_wavelet
from utils import iterate_conditions, check_paths
import scipy
from mne.stats import combine_adjacency
from mne.stats import fdr_correction

%matplotlib qt

**ONE CONDITION**

Fixes for later:
- one scale for all heatmaps
- check why Y_BL_go alpha-gamma is all zero 

In [15]:
groups = ['Y', 'O']
tasks = ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
fig_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\s2_results'

# Topomap settings
window = 0.100   # 100 ms
step = 0.050     # 50 ms
start, end = 0.0, 0.5
n_rows, n_cols = 3, 3

time_slices = []
t = start
while t + window <= end:
    time_slices.append((t, t + window))
    t += step


for group, task, task_stage, block in iterate_conditions(groups, tasks, task_stages):
    print('Processing:', group, task, task_stage, block, '...')
    group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
    sub_name = 'ALL_subs'

    epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block}_ALL-epo.fif"), preload=True)
    epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))

    ####### PAC computation #######

    # --- Select EEG channels only ---
    epochs_eeg = epochs_all_subs.copy().pick("eeg")
    epochs_eeg = epochs_eeg.crop(tmin=-0.05, tmax=0.5)

    # --- Pre-allocate ---
    n_channels = len(epochs_eeg.ch_names)
    n_times = len(epochs_eeg.times)

    sf = epochs_eeg.info['sfreq']
    epo_data = epochs_eeg.get_data()
    time = epochs_eeg.times
    info = epochs_eeg.info
    alpha_threshold = 0.05

    # --- Loop over phase freqs and channels ---
    theta = [4, 8]
    alpha = [8, 12]
    gamma = (30, 80, 5, 1)

    for phase_freq in [theta, alpha]:
        if phase_freq == theta:
            pac_name = 'tgPAC'
        else:
            pac_name = 'agPAC'
        
        all_sig_mean = np.zeros((n_channels, n_times))
        p = EventRelatedPac(f_pha=phase_freq, f_amp=gamma) # theta [4, 8], alpha [8, 12], gamma [30, 80]
        n_freqs = len(p.yvec)
        all_erpac = np.zeros((n_channels, n_freqs, n_times))

        for ch_idx, ch_name in enumerate(epochs_eeg.ch_names):
            x = epo_data[:, ch_idx, :]

            # Compute ERPAC for this channel
            erpac = p.filterfit(sf, x, method='circular', mcp='bonferroni').squeeze()
            pvalues = p.pvalues.squeeze()

            # Compute mean significant PAC across frequencies for each time point
            sig_mask = pvalues <= alpha_threshold
            sig_mean = np.where(
                np.any(sig_mask, axis=0),
                np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
                0
            )

            all_erpac[ch_idx, :, :] = erpac
            all_sig_mean[ch_idx, :] = sig_mean


        ####### Plot PAC #######
    
        ####### 1. As heatmap #######
        fig = plt.figure(figsize=(19, 10))
        im = plt.imshow(
            all_sig_mean,
            aspect='auto',
            extent=[time[0], time[-1], 0, n_channels],
            cmap='Spectral_r',
            origin='lower'
        )

        plt.colorbar(im, label='Mean significant ERPAC')
        plt.yticks(np.arange(n_channels) + 0.5, epochs_eeg.ch_names)
        plt.xlabel('Time (s)')
        plt.ylabel('Channel')
        plt.title(f'{group}{task}{task_stage}{block}: Time-resolved mean significant {pac_name} across channels')
        plt.tight_layout()
        plt.show()
        # --- Save figure ---
        fig_save_path = os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_time_resolved_mean_sig_{pac_name}_heatmap.png")
        fig.savefig(fig_save_path)


        ####### 2. As topomap #######
        highlight_per_window = []  # list of lists of electrode indices

        for tmin, tmax in time_slices:
            tidx = np.where((time >= tmin) & (time < tmax))[0]

            sig_in_window = np.any(all_sig_mean[:, tidx] != 0, axis=1)
            # True at channels that were significant

            highlight_per_window.append(np.where(sig_in_window)[0])

        # Figure with topomap subplots
        # 3 rows, 3 columns
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(19, 10))
        # Flatten axes array for easy iteration
        axes = axes.flatten()
        # Set lims for color scale
        lim = np.nanmax(np.abs(all_sig_mean))
        global_vmin, global_vmax = -0.03, lim

        for ax, ((tmin, tmax), highlight) in zip(axes, zip(time_slices, highlight_per_window)):

            tidx = np.where((time >= tmin) & (time < tmax))[0]
            window_data = np.nanmean(all_sig_mean[:, tidx], axis=1)

            # Create mask for significant electrodes
            mask = np.zeros(window_data.shape, dtype=bool)
            mask[highlight] = True

            # Plot topomap in the given subplot
            im_topo, _ = mne.viz.plot_topomap(
                data=window_data,
                pos=info,
                axes=ax,
                vlim=(global_vmin, global_vmax),
                show=False,
                cmap="magma",
                mask=mask,
                mask_params=dict(marker='o', markerfacecolor='white', markersize=6),
                outlines="head"
            )

            ax.set_title(f"{tmin*1000:.0f}-{tmax*1000:.0f} ms", fontsize=10)

        # Remove any extra subplots if time_slices < n_rows*n_cols
        for ax in axes[len(time_slices):]:
            ax.axis('off')
        # ---- ADD SINGLE COLORBAR ----
        # Leave space on the right (adjust this value depending on the cbar width)
        plt.subplots_adjust(right=0.88)
        # Add a colorbar axis at far right
        cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
        fig.colorbar(im_topo, cax=cbar_ax, label='Mean significant ERPAC')
        fig.suptitle(f'{group}{task}{task_stage}{block}: {pac_name}')
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

        # save figs and data
        fig.savefig(os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_{pac_name}_topomap.png"), dpi=300)
        np.save(os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_{pac_name}_all_sig_mean.npy"), all_sig_mean)
        plt.close('all')


Processing: O _BL _plan  ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O group\O_ALL_subs_BL_epochs_plan_ALL-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 13 columns
2431 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)


Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
C:\Users\a1902989\AppData\Local\Temp\ipykernel_18484\1431601611.py:70: RuntimeWarning: Mean of empty slice
  np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=4

Processing: O _BL _go  ...
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O group\O_ALL_subs_BL_epochs_go_ALL-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 13 columns
2283 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)


Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
C:\Users\a1902989\AppData\Local\Temp\ipykernel_18484\1431601611.py:70: RuntimeWarning: Mean of empty slice
  np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
    Extract phases (n_pha=1) and amplitudes (n_amps=4

Stop within-condition analysis here and cump to GROUP COMPARISON section

_____________________________

In [29]:
im_topo, _ = mne.viz.plot_topomap(
                data=window_data,
                pos=info,
                axes=ax,
                vlim=(global_vmin, global_vmax),
                show=False,
                cmap="magma",
                mask=mask,
                mask_params=dict(marker='o', markerfacecolor='white', markersize=6),
                outlines="head"
            )

ax.set_title(f"{tmin*1000:.0f}-{tmax*1000:.0f} ms", fontsize=10)

# Remove any extra subplots if time_slices < n_rows*n_cols
for ax in axes[len(time_slices):]:
    ax.axis('off')
# ---- ADD SINGLE COLORBAR ----
cbar = fig.colorbar(im_topo, ax=axes.tolist(), shrink=0.7)
cbar.set_label("Mean significant ERPAC")

fig.suptitle(f'{group}{task}{task_stage}{block}: {pac_name}')
plt.tight_layout(rect=[0, 0, 0.95, 0.95])
plt.show()

In [5]:
test_fig_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\s2_results\\Y_BL_plan'
print(test_fig_dir)

D:\BonoKat\research project\# study 1\eeg_data\set\figures\s2_results\Y_BL_plan


In [24]:
group = 'Y'
group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
figs_dir = os.path.join(group_save_path, 'sensors', 'figs', 'TF')
sub_name = 'ALL_subs'
task = '_BL'
task_stage = '_plan'
block_name = ''
epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))
epochs_all_subs

Reading D:\BonoKat\research project\# study 1\eeg_data\set\Y group\Y_ALL_subs_BL_epochs_plan_ALL-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 13 columns
2534 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)


<EpochsFIF | 2534 events (all good), -0.5 – 0.5 s (baseline -0.5 – -0.001 s), ~590.9 MiB, data loaded, with metadata,
 'prepare': 2534>

One-channel PAC

In [ ]:
# One channel PAC with plotting
epo_data = epochs_all_subs.get_data()

for ch_idx in range(len(epochs_all_subs.ch_names))[6:7]:
    # define an ERPAC object
    p = EventRelatedPac(f_pha=[3, 8], f_amp=(30, 80, 5, 1))
    sf = epochs_all_subs.info['sfreq']  # sampling frequency
    time = epochs_all_subs.times  # time vector
    x = epo_data[:, ch_idx, :]  # shape is (n_epochs, n_times)

    # method for correcting p-values for multiple comparisons
    mcp = 'bonferroni'
    # extract phases and amplitudes
    erpac = p.filterfit(sf, x, method='circular', mcp=mcp, n_perm=10000).squeeze()
    # get the p-values and squeeze unused dimensions
    pvalues = p.pvalues.squeeze()
    # set to nan everywhere it's not significant
    erpac[pvalues > .05] = np.nan

    vmin, vmax = np.nanmin(erpac), np.nanmax(erpac)
    
    plt.figure(figsize=(6, 4))
    fig = p.pacplot(erpac, time, p.yvec, xlabel='Time (second)',
            cmap='Spectral_r', ylabel='Amplitude frequency', title=f'ERPAC: {epochs_all_subs.ch_names[ch_idx]}',
            cblabel='ERPAC', rmaxis=True, vmin=vmin, vmax=vmax)
    plt.axvline(1., linestyle='--', color='k', linewidth=2)

    plt.tight_layout()
    plt.show()


# ----- Significance threshold -----
alpha = 0.05
sig_mask = pvalues <= alpha   # True = significant

# ----- Compute mean significant PAC at each time point -----
# erpac.shape = (n_freqs, n_times)
sig_mean = np.zeros(erpac.shape[1])  # initialize output vector

# Alternatively (vectorized approach):
sig_mean = np.where(
    np.any(sig_mask, axis=0),
    np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0),
    0
)

# ----- Plot -----
time = epochs_all_subs.times

plt.figure(figsize=(6, 3))
plt.plot(time, sig_mean, color='darkred', linewidth=2)
plt.axvline(1., linestyle='--', color='k', linewidth=1.5)
plt.xlabel('Time (s)')
plt.ylabel('Mean significant ERPAC')
plt.title(f'Mean significant PAC over time: {epochs_all_subs.ch_names[ch_idx]}')
plt.tight_layout()
plt.show()

Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
C:\Users\a1902989\AppData\Local\Temp\ipykernel_40564\2698311760.py:43: RuntimeWarning: Mean of empty slice
  np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0),


PAC: all on one plot

In [25]:
# --- Select EEG channels only ---
epochs_eeg = epochs_all_subs.copy().pick("eeg")
epochs_eeg = epochs_eeg.crop(tmin=-0.05, tmax=0.5)

# Phase freqs
theta = [4, 8]
alpha = [8, 12]

# --- Pre-allocate ---
n_channels = len(epochs_eeg.ch_names)
n_times = len(epochs_eeg.times)
p_frame = EventRelatedPac(f_pha=alpha, f_amp=(30, 80, 5, 1))
n_freqs = len(p_frame.yvec)
all_erpac = np.zeros((n_channels, n_freqs, n_times))
all_sig_mean = np.zeros((n_channels, n_times))

sf = epochs_eeg.info['sfreq']
epo_data = epochs_eeg.get_data()
time = epochs_eeg.times
alpha_threshold = 0.05

# --- Loop over channels ---
for ch_idx, ch_name in enumerate(epochs_eeg.ch_names):
    x = epo_data[:, ch_idx, :]

    # Compute ERPAC for this channel
    p = EventRelatedPac(f_pha=alpha, f_amp=(30, 80, 5, 1)) # theta [4, 8], alpha [8, 12], gamma [30, 80]
    erpac = p.filterfit(sf, x, method='circular', mcp='bonferroni').squeeze()
    pvalues = p.pvalues.squeeze()

    # Compute mean significant PAC across frequencies for each time point
    sig_mask = pvalues <= alpha_threshold
    sig_mean = np.where(
        np.any(sig_mask, axis=0),
        np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
        0
    )

    all_erpac[ch_idx, :, :] = erpac
    all_sig_mean[ch_idx, :] = sig_mean

# --- Plot as heatmap ---
plt.figure(figsize=(8, 6))
im = plt.imshow(
    all_sig_mean,
    aspect='auto',
    extent=[time[0], time[-1], 0, n_channels],
    cmap='Spectral_r',
    origin='lower'
)

plt.colorbar(im, label='Mean significant ERPAC')
plt.yticks(np.arange(n_channels) + 0.5, epochs_eeg.ch_names)
plt.xlabel('Time (s)')
plt.ylabel('Channel')
plt.title('Time-resolved mean significant PAC across channels')
plt.tight_layout()
plt.show()


Event Related PAC object defined
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
C:\Users\a1902989\AppData\Local\Temp\ipykernel_40564\967486229.py:35: RuntimeWarning: Mean of empty slice
  np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correct p-values for multiple-comparisons using bonferroni correction of MNE-Python
Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute ERPAC (Voytek et al. 2013)
    Correc

In [35]:
all_erpac.shape

(60, 45, 276)

In [36]:
all_sig_mean.shape

(60, 276)

Topoplots

In [4]:
all_sig_mean.shape

(60, 276)

In [5]:
window = 0.100   # 100 ms
step = 0.050     # 50 ms
start, end = 0.0, 0.5

time_slices = []
t = start
while t + window <= end:
    time_slices.append((t, t + window))
    t += step

print(time_slices)


[(0.0, 0.1), (0.05, 0.15000000000000002), (0.1, 0.2), (0.15000000000000002, 0.25), (0.2, 0.30000000000000004), (0.25, 0.35), (0.3, 0.4), (0.35, 0.44999999999999996), (0.39999999999999997, 0.5)]


In [6]:
# sig_mean: array (n_channels, n_times)
# times: the MNE time vector e.g. epochs.times

highlight_per_window = []  # list of lists of electrode indices

for tmin, tmax in time_slices:
    tidx = np.where((time >= tmin) & (time < tmax))[0]

    sig_in_window = np.any(all_sig_mean[:, tidx] != 0, axis=1)
    # True at channels that were significant

    highlight_per_window.append(np.where(sig_in_window)[0])


In [7]:
highlight_per_window

[array([ 2,  4,  6,  9, 19, 22, 28, 55, 57]),
 array([ 2,  4,  6,  7,  8,  9, 14, 15, 17, 19, 22, 28, 34, 38, 40, 46, 47,
        54, 55, 56, 57, 59]),
 array([ 0,  1,  2,  4,  6,  7,  8,  9, 11, 14, 15, 17, 19, 22, 25, 32, 34,
        38, 40, 41, 46, 47, 52, 54, 55, 56, 58, 59]),
 array([ 0,  1,  6,  7,  8,  9, 11, 14, 15, 17, 19, 25, 32, 35, 38, 40, 41,
        46, 47, 48, 52, 53, 54, 55, 56, 58, 59]),
 array([ 6,  8, 11, 14, 15, 25, 35, 48, 53, 58]),
 array([ 8, 35, 53, 58]),
 array([], dtype=int64),
 array([25]),
 array([25])]

In [ ]:
# Figure with topomap subplots
pac_type = 'tg' # 'ag'
info = epochs_eeg.info

# 2 rows, 5 columns
n_rows, n_cols = 3, 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))

# Flatten axes array for easy iteration
axes = axes.flatten()

for ax, ((tmin, tmax), highlight) in zip(axes, zip(time_slices, highlight_per_window)):

    tidx = np.where((time >= tmin) & (time < tmax))[0]
    window_data = np.nanmean(all_sig_mean[:, tidx], axis=1)

    # Create mask for significant electrodes
    mask = np.zeros(window_data.shape, dtype=bool)
    mask[highlight] = True

    # Plot topomap in the given subplot
    mne.viz.plot_topomap(
        data=window_data,
        pos=info,
        axes=ax,
        show=False,
        cmap="viridis",
        mask=mask,
        mask_params=dict(marker='o', markerfacecolor='white', markersize=6),
        outlines="head"
    )

    ax.set_title(f"{tmin*1000:.0f}-{tmax*1000:.0f} ms", fontsize=10)

# Remove any extra subplots if time_slices < n_rows*n_cols
for ax in axes[len(time_slices):]:
    ax.axis('off')

plt.tight_layout()
plt.show()
plt.savefig(os.path.join(test_fig_dir, f"{pac_type}PAC_pac_topomap_fig.png"), dpi=300)

In [ ]:
# GIF of topomaps
import imageio.v2 as imageio

fig.canvas.draw()
renderer = fig.canvas.get_renderer()
image = np.array(renderer.buffer_rgba())[:, :, :3]  # take RGB channels

images = []

for (tmin, tmax), highlight in zip(time_slices, highlight_per_window):

    tidx = np.where((time >= tmin) & (time < tmax))[0]
    window_data = np.nanmean(all_sig_mean[:, tidx], axis=1)

    mask = np.zeros(window_data.shape, dtype=bool)
    mask[highlight] = True

    fig, ax = plt.subplots(figsize=(4, 4))

    mne.viz.plot_topomap(
        data=window_data,
        pos=info,
        axes=ax,
        show=False,
        cmap="viridis",
        mask=mask,
        mask_params=dict(marker='o', markerfacecolor='white',
                         markersize=12),
        outlines="head"
    )

    ax.set_title(f"{tmin*1000:.0f}–{tmax*1000:.0f} ms")

    fig.canvas.draw()

    # Convert figure to NumPy array (works on all backends)
    renderer = fig.canvas.get_renderer()
    image = np.array(renderer.buffer_rgba())[:, :, :3]  # RGB only

    images.append(image)
    plt.close(fig)

# Save as GIF
imageio.mimsave(os.path.join(test_fig_dir, f"{pac_type}PAC_pac_topomap.gif"), images, fps=1)

_________________________________________________

# GROUP COMPARISON

**Question:**
Is PAC different between conditions? -> surrogates are based on swapping condition labels for PAC values (phase and amp freqs stay unchanged)

In [157]:
# Conditions definition: name -> (group, task, stage, block)
conditions = {
    # --- Baseline (_BL) ---
    "Y_BL_go":      ("Y", "_BL", "_go", None),
    "Y_BL_plan":    ("Y", "_BL", "_plan", None),

    "O_BL_go":      ("O", "_BL", "_go", None),
    "O_BL_plan":    ("O", "_BL", "_plan", None),

    # --- MAIN baseline ---
    "Y_MAIN_go_baseline":   ("Y", "_MAIN", "_go", "_baseline"),
    "Y_MAIN_plan_baseline": ("Y", "_MAIN", "_plan", "_baseline"),

    "O_MAIN_go_baseline":   ("O", "_MAIN", "_go", "_baseline"),
    "O_MAIN_plan_baseline": ("O", "_MAIN", "_plan", "_baseline"),

    # --- MAIN adaptation ---
    "Y_MAIN_go_adaptation":   ("Y", "_MAIN", "_go", "_adaptation"),
    "Y_MAIN_plan_adaptation": ("Y", "_MAIN", "_plan", "_adaptation"),

    "O_MAIN_go_adaptation":   ("O", "_MAIN", "_go", "_adaptation"),
    "O_MAIN_plan_adaptation": ("O", "_MAIN", "_plan", "_adaptation"),

    # --- MAIN combined (both blocks) ---
    "Y_MAIN_go_combined":   ("Y", "_MAIN", "_go", None),
    "Y_MAIN_plan_combined": ("Y", "_MAIN", "_plan", None),

    "O_MAIN_go_combined":   ("O", "_MAIN", "_go", None),
    "O_MAIN_plan_combined": ("O", "_MAIN", "_plan", None),
}

comparisons = [
    # --- Baseline (BL) ---
    ("O_BL_plan", "Y_BL_plan"),
    ("O_BL_go",   "Y_BL_go"),

    # --- MAIN baseline ---
    ("O_MAIN_plan_baseline", "Y_MAIN_plan_baseline"),
    ("O_MAIN_go_baseline",   "Y_MAIN_go_baseline"),

    # --- MAIN adaptation ---
    ("O_MAIN_plan_adaptation", "Y_MAIN_plan_adaptation"),
    ("O_MAIN_go_adaptation",   "Y_MAIN_go_adaptation"),
]


In [ ]:
groups = ['Y', 'O']
tasks = ['_BL', '_MAIN']
task_stages = ['_plan', '_go']
fig_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\s2_results'

# Topomap settings
window = 0.100   # 100 ms
step = 0.050     # 50 ms
start, end = 0.0, 0.5
n_rows, n_cols = 3, 3

time_slices = []
t = start
while t + window <= end:
    time_slices.append((t, t + window))
    t += step


for group, task, task_stage, block in iterate_conditions(groups, tasks, task_stages):
    print('Processing:', group, task, task_stage, block, '...')
    group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
    sub_name = 'ALL_subs'

    epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block}_ALL-epo.fif"), preload=True)
    epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))

    ####### PAC computation #######

    # --- Select EEG channels only ---
    epochs_eeg = epochs_all_subs.copy().pick("eeg")
    epochs_eeg = epochs_eeg.crop(tmin=-0.05, tmax=0.5)

    # --- Pre-allocate ---
    n_channels = len(epochs_eeg.ch_names)
    n_times = len(epochs_eeg.times)

    sf = epochs_eeg.info['sfreq']
    epo_data = epochs_eeg.get_data()
    time = epochs_eeg.times
    info = epochs_eeg.info
    alpha_threshold = 0.05

    # --- Loop over phase freqs and channels ---
    theta = [4, 8]
    alpha = [8, 12]
    gamma = (30, 80, 5, 1)

    for phase_freq in [theta, alpha]:
        if phase_freq == theta:
            pac_name = 'tgPAC'
        else:
            pac_name = 'agPAC'
        
        all_sig_mean = np.zeros((n_channels, n_times))
        p = EventRelatedPac(f_pha=phase_freq, f_amp=gamma) # theta [4, 8], alpha [8, 12], gamma [30, 80]
        n_freqs = len(p.yvec)
        all_erpac = np.zeros((n_channels, n_freqs, n_times))

        for ch_idx, ch_name in enumerate(epochs_eeg.ch_names):
            x = epo_data[:, ch_idx, :]

            # Compute ERPAC for this channel
            erpac = p.filterfit(sf, x, method='circular', mcp='bonferroni').squeeze()
            pvalues = p.pvalues.squeeze()

            # Compute mean significant PAC across frequencies for each time point
            sig_mask = pvalues <= alpha_threshold
            sig_mean = np.where(
                np.any(sig_mask, axis=0),
                np.nanmean(np.where(sig_mask, erpac, np.nan), axis=0), # npmean or npmax
                0
            )

            all_erpac[ch_idx, :, :] = erpac
            all_sig_mean[ch_idx, :] = sig_mean


        ####### Plot PAC #######
    
        ####### 1. As heatmap #######
        fig = plt.figure(figsize=(19, 10))
        im = plt.imshow(
            all_sig_mean,
            aspect='auto',
            extent=[time[0], time[-1], 0, n_channels],
            cmap='Spectral_r',
            origin='lower'
        )

        plt.colorbar(im, label='Mean significant ERPAC')
        plt.yticks(np.arange(n_channels) + 0.5, epochs_eeg.ch_names)
        plt.xlabel('Time (s)')
        plt.ylabel('Channel')
        plt.title(f'{group}{task}{task_stage}{block}: Time-resolved mean significant {pac_name} across channels')
        plt.tight_layout()
        plt.show()
        # --- Save figure ---
        fig_save_path = os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_time_resolved_mean_sig_{pac_name}_heatmap.png")
        fig.savefig(fig_save_path)


        ####### 2. As topomap #######
        highlight_per_window = []  # list of lists of electrode indices

        for tmin, tmax in time_slices:
            tidx = np.where((time >= tmin) & (time < tmax))[0]

            sig_in_window = np.any(all_sig_mean[:, tidx] != 0, axis=1)
            # True at channels that were significant

            highlight_per_window.append(np.where(sig_in_window)[0])

        # Figure with topomap subplots
        # 3 rows, 3 columns
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(19, 10))
        # Flatten axes array for easy iteration
        axes = axes.flatten()
        # Set lims for color scale
        lim = np.nanmax(np.abs(all_sig_mean))
        global_vmin, global_vmax = -0.03, lim

        for ax, ((tmin, tmax), highlight) in zip(axes, zip(time_slices, highlight_per_window)):

            tidx = np.where((time >= tmin) & (time < tmax))[0]
            window_data = np.nanmean(all_sig_mean[:, tidx], axis=1)

            # Create mask for significant electrodes
            mask = np.zeros(window_data.shape, dtype=bool)
            mask[highlight] = True

            # Plot topomap in the given subplot
            im_topo, _ = mne.viz.plot_topomap(
                data=window_data,
                pos=info,
                axes=ax,
                vlim=(global_vmin, global_vmax),
                show=False,
                cmap="magma",
                mask=mask,
                mask_params=dict(marker='o', markerfacecolor='white', markersize=6),
                outlines="head"
            )

            ax.set_title(f"{tmin*1000:.0f}-{tmax*1000:.0f} ms", fontsize=10)

        # Remove any extra subplots if time_slices < n_rows*n_cols
        for ax in axes[len(time_slices):]:
            ax.axis('off')
        # ---- ADD SINGLE COLORBAR ----
        # Leave space on the right (adjust this value depending on the cbar width)
        plt.subplots_adjust(right=0.88)
        # Add a colorbar axis at far right
        cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
        fig.colorbar(im_topo, cax=cbar_ax, label='Mean significant ERPAC')
        fig.suptitle(f'{group}{task}{task_stage}{block}: {pac_name}')
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

        # save figs and data
        fig.savefig(os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_{pac_name}_topomap.png"), dpi=300)
        np.save(os.path.join(fig_dir, f"{group}{task}{task_stage}{block}_{pac_name}_all_sig_mean.npy"), all_sig_mean)
        plt.close('all')


In [ ]:
group = 'Y'
group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
figs_dir = os.path.join(group_save_path, 'sensors', 'figs', 'TF')
cond_dir = os.path.join(figs_dir, 'conditions')
check_paths(cond_dir)

sub_name = 'ALL_subs'
task = '_BL'
task_stage = '_plan'
block_name = ''
epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block_name}_ALL-epo.fif"), preload=True)
epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))
epochs_all_subs

In [142]:
def extract_data_for_pac(group, task, task_stage, block):
    if block is None:
        block = ''
    group_save_path = f'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\{group} group'
    sub_name = 'ALL_subs'

    epochs_all_subs = mne.read_epochs(os.path.join(group_save_path, f"{group}_{sub_name}{task}_epochs{task_stage}{block}_ALL-epo.fif"), preload=True)
    epochs_all_subs.apply_baseline(baseline=(-0.5, -0.001))

    # --- Select EEG channels only ---
    epochs_eeg = epochs_all_subs.copy().pick("eeg")
    epochs_eeg = epochs_eeg.crop(tmin=-0.05, tmax=0.5)
    epo_data = epochs_eeg.get_data()

    return epochs_eeg, epo_data

In [143]:
def one_perm(combined, idx, n1):
    pseudo1 = combined[idx[:n1]]
    pseudo2 = combined[idx[n1:]]
    er1 = p.filterfit(sf, pseudo1, method="gc", n_perm=0)
    er2 = p.filterfit(sf, pseudo2, method="gc", n_perm=0)
    return er1 - er2


Cluster-based permutation stats

In [144]:
def load_epochs_for_group(sub_list, group, eeg_path, task, stage, block,
                          baseline=(-0.5, -0.001), crop_t=(-0.05, 0.5)):
    """
    Load, baseline, pick EEG, crop, and extract epoch data for a list of subjects.

    Parameters
    ----------
    sub_list : list of str
        Subject IDs.
    group : str
        Group folder name.
    eeg_path : str
        Base path to EEG data.
    task, stage, block : str
        Strings used to build filename: f"{sub}{task}_epochs{stage}{block}-epo.fif".
    baseline : tuple
        Baseline period applied with `apply_baseline()`.
    crop_t : tuple
        (tmin, tmax) time window for `crop()`.

    Returns
    -------
    list of ndarray
        List of epoch arrays (n_epochs, n_channels, n_times) for each subject.
    """
    epochs_list = []
    for sub in sub_list:
        fname = os.path.join(
            eeg_path, group, sub, 'preproc', 'analysis',
            f"{sub}{task}_epochs{stage}{block}-epo.fif"
        )
        epochs = mne.read_epochs(fname, preload=True)
        epochs.apply_baseline(baseline)
        epochs_eeg = epochs.copy().pick("eeg")
        epochs_eeg.crop(tmin=crop_t[0], tmax=crop_t[1])
        epochs_list.append(epochs_eeg.get_data())
    return epochs_list, epochs_eeg


In [50]:
def compute_erpac_two_groups(epochs_group1, epochs_group2, sf, phase_freqs, amp_freqs):
    """
    Compute ERPAC for two conditions across channels and phase frequencies.

    Parameters
    ----------
    epochs_eeg1 : ndarray
        Shape (n_epochs1, n_channels, n_times)
    epochs_eeg2 : ndarray
        Shape (n_epochs2, n_channels, n_times)
    sf : float
        Sampling frequency.
    phase_freqs : list of arrays
        List of frequency ranges, e.g. [theta] or [theta, alpha].
    gamma : array
        Amplitude frequency range for ERPAC.

    Returns
    -------
    dict
        pac_name -> results dict with:
            - all_erpac1
            - all_erpac2
            - n_freqs
            - n_channels
            - n_times
    """

    results = {}

    n_channels = epochs_group1.shape[1]
    n_times = epochs_group1.shape[2]

    # --- Loop over phase freqs (theta, alpha, etc.) ---
    for phase_freq in phase_freqs:

        pac_name = 'tg' if np.allclose(phase_freq, [4, 8]) else 'ag'

        # PAC object
        p = EventRelatedPac(f_pha=phase_freq, f_amp=amp_freqs)
        n_freqs = len(p.yvec)

        erpac_all_1 = np.zeros((n_channels, n_freqs, n_times))
        erpac_all_2 = np.zeros((n_channels, n_freqs, n_times))

        # --- Loop over channels ---
        for ch_idx in range(n_channels):

            cond1_data = epochs_group1[:, ch_idx, :]
            cond2_data = epochs_group2[:, ch_idx, :]

            # Compute ERPAC without permutations
            erpac1 = p.filterfit(sf, cond1_data, method="gc", n_perm=0)
            erpac2 = p.filterfit(sf, cond2_data, method="gc", n_perm=0)

            erpac_all_1[ch_idx] = erpac1.squeeze()
            erpac_all_2[ch_idx] = erpac2.squeeze()

        # Store results for this PAC type
        results[pac_name] = {
            "erpac_group1": erpac_all_1,
            "erpac_group2": erpac_all_2,
            'PAC_name': pac_name,
            "phase_freqs": phase_freq,
            "amp_freqs": amp_freqs,
            "n_freqs": n_freqs,
            "n_channels": n_channels,
            "n_times": n_times,
        }

    return results


In [145]:
def compute_erpac_subject(epochs_eeg, sf, phase_freqs, amp_freqs):
    """
    Compute ERPAC per subject.

    Parameters
    ----------
    epochs_eeg : ndarray
        Shape (n_epochs, n_channels, n_times)
    sf : float
        Sampling frequency.
    phase_freqs : list of arrays (phase-frequency bands)
    gamma : array
        Amplitude-frequency band.

    Returns
    -------
    dict
        pac_name -> array with shape (n_channels, n_freqs, n_times)
    """
    results = {}

    n_channels = epochs_eeg.shape[1]
    n_times = epochs_eeg.shape[2]

    for phase_freq in phase_freqs:

        pac_name = 'tgPAC' if np.allclose(phase_freq, [4, 8]) else 'agPAC'
        p = EventRelatedPac(f_pha=phase_freq, f_amp=amp_freqs)
        n_freqs = len(p.yvec)

        erpac_all = np.zeros((n_channels, n_freqs, n_times))

        for ch_idx in range(n_channels):
            data = epochs_eeg[:, ch_idx, :]   # shape: (n_epochs, n_times)
            erpac = p.filterfit(sf, data, method="gc", n_perm=0)
            erpac_all[ch_idx] = erpac.squeeze()

        results[pac_name] = erpac_all

    return results


In [ ]:
# DONE for theta-gamma PAC comparison between groups

eeg_path = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set'
stats_save_path = os.path.join('D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\figures\\s2_results', 'stats', 'conditions')
figs_dir = os.path.join(stats_save_path, 'figures')
check_paths(stats_save_path, figs_dir)

groups = ['Y', 'O']
subs = [os.listdir(os.path.join(eeg_path, group)) for group in groups]

theta = [4, 8]
# alpha = [8, 12]
gamma = (30, 80, 5, 1)

for cond1, cond2 in comparisons:

    ########### 1. Compute ERPAC for both groups ###########
    print('Processing comparison:', cond1, 'vs', cond2)
    group1, task1, stage1, block1 = conditions[cond1]
    group2, task2, stage2, block2 = conditions[cond2]
    block1, block2 = block1 or '', block2 or ''

    epochs_all_data1, epochs = load_epochs_for_group(subs[1], group1, eeg_path, task1, stage1, block1) # older group
    epochs_all_data2 = load_epochs_for_group(subs[0], group2, eeg_path, task2, stage2, block2)[0] # younger group

    # --- Pre-allocate ---
    n_channels = len(epochs.ch_names)
    n_times = len(epochs.times)
    sf = epochs.info['sfreq']
    time = epochs.times
    info = epochs.info

    group1_results = []
    for subj_data in epochs_all_data1:
        group1_results.append(compute_erpac_subject(subj_data, sf, phase_freqs=[theta], amp_freqs=gamma)) # CURRENTLY: theta-gamma PAC

    group2_results = []
    for subj_data in epochs_all_data2:
        group2_results.append(compute_erpac_subject(subj_data, sf, phase_freqs=[theta], amp_freqs=gamma))
    
    ############# 2. Run cluster-based statistical comparison ###############
    pac_name = list(group1_results[0].keys())[0] #! CHECK if alpha-gamma or theta-gamma
    g1 = np.transpose(np.stack([r['tgPAC'] for r in group1_results]), axes=(0, 3, 2, 1))
    g2 = np.transpose(np.stack([r['tgPAC'] for r in group2_results]), axes=(0, 3, 2, 1))
    g1_ave = np.mean(g1, axis=(2)) # (24, 276, 45, 60) -> (24, 276, 60) subs x times x electrodes
    g2_ave = np.mean(g2, axis=(2))

    X = [g1_ave, g2_ave]  # group 1 and group 2 subject-level PAC

    # Create channel adjacency matrix
    sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, "eeg")

    n_times = X[0].shape[1]    # 276
    # n_freqs = X[0].shape[2]    # 45
    n_chans = X[0].shape[2]    # 60

    adjacency = combine_adjacency(n_times, sensor_adjacency)
    print(f'Adjacency matrix shape: {adjacency.shape}')

    degrees_of_freedom = g1.shape[0] + g2.shape[0] - 2
    t_thresh = scipy.stats.t.ppf(1 - 0.01 / 2, df=degrees_of_freedom)
    tail = 0 # two-tailed test

    # Set the number of permutations
    n_permutations = 10000

    T_obs, clusters, cluster_p, H0 = mne.stats.spatio_temporal_cluster_test(
        X,
        n_permutations=n_permutations, 
        threshold=t_thresh,
        tail=tail,
        adjacency=adjacency,
        out_type="mask",
        n_jobs=-1
    )

    # SANITY CHECKS
    print(f'T_obs_mean = {T_obs.mean()}')
    print(f'cluster_p_values = {cluster_p}')

    alpha_threshold = 0.05  # significance threshold
    significant_clusters = [i for i, p in enumerate(cluster_p) if p < alpha_threshold]
    print(f"{cond1} vs {cond2} found {len(significant_clusters)} significant clusters")

    # # Save the results
    np.save(os.path.join(stats_save_path, f"{cond1}_{pac_name}.npy"), g1)
    np.save(os.path.join(stats_save_path, f"{cond2}_{pac_name}.npy"), g2)
    np.save(os.path.join(stats_save_path, f"{cond1}_vs_{cond2}_freqs_ave_T_obs.npy"), T_obs)
    np.save(os.path.join(stats_save_path, f"{cond1}_vs_{cond2}_freqs_ave_clusters.npy"), np.array(clusters, dtype=object))
    np.save(os.path.join(stats_save_path, f"{cond1}_vs_{cond2}_freqs_ave_cluster_p_values.npy"), cluster_p)


    ####### 3. Plot results #######
    ### A. Highlight clusters on top of T-map

    plt.figure(figsize=(19, 10))
    plt.imshow(T_obs.T, aspect='auto', origin='lower', cmap='plasma', vmin=0, vmax=8)
    plt.colorbar(label='T-value')

    # overlay cluster outlines
    for i, p_val in enumerate(cluster_p):
        if p_val < 0.05:
            mask = clusters[i]
            plt.contour(mask.T, levels=[0.5], colors='white', linewidths=1.5)

    plt.yticks(
        ticks=np.arange(n_channels),
        labels=ch_names
    )

    plt.xticks(
        ticks=np.linspace(0, len(epochs.times) - 1, 8),   # 8 evenly spaced ticks
        labels=np.round(np.linspace(epochs.times[0], epochs.times[-1], 8), 2)  # seconds
    )
        
    plt.title(f"Significant {pac_name} clusters: {cond1} vs {cond2}")
    plt.xlabel("Time (s)")
    plt.ylabel("Channel")
    plt.show()

    # Save topomap figures and data for each condition
    plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_sig_cluster_heatmap.png"), dpi=300)

    ### B. Plot the cluster-averaged timecourse
    sig_clusters = [clusters[i] for i, p in enumerate(cluster_p) if p < alpha_threshold]

    if len(sig_clusters) > 0:
        mask = sig_clusters[0]   # first cluster
        time_mask = mask.any(axis=1)      # which time points
        chan_mask = mask.any(axis=0)      # which channels
        true_times = epochs.times[time_mask]

        g1_mean = g1_ave.mean(axis=0)  # (times, channels)
        g2_mean = g2_ave.mean(axis=0)

        g1_cluster_ts = g1_mean[time_mask][:, chan_mask].mean(axis=1)
        g2_cluster_ts = g2_mean[time_mask][:, chan_mask].mean(axis=1)

        plt.figure(figsize=(8, 4))
        plt.plot(g1_cluster_ts, label="O group")
        plt.plot(g2_cluster_ts, label="Y group")
        plt.legend()

        plt.xticks(
        ticks=np.linspace(0, len(true_times) - 1, 8),   # 8 evenly spaced ticks
        labels=np.round(np.linspace(true_times[0], true_times[-1], 8), 2)  # seconds
        )
        plt.title(f"Average {pac_name} within significant cluster: {cond1} vs {cond2}")
        plt.xlabel("Time (s)")
        plt.ylabel("PAC")
        plt.show()

        # Save figure
        plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_ave_sig_cluster_timecourse.png"), dpi=300)
    
    ### C. Plot topomap with significant electrodes highlighted
    # Combine all significant clusters
    sig_mask = np.zeros_like(clusters[0], dtype=bool)
    for i, p_val in enumerate(cluster_p):
        if p_val < alpha_threshold:
            sig_mask |= clusters[i]

    # Find first significant time index
    first_sig_idx = np.where(sig_mask.any(axis=1))[0][0]

    # Define sliding windows
    sf = 1 / (times[1] - times[0])  # sampling frequency from time vector
    win_size = int(0.1 * sf)        # 100 ms
    step_size = int(0.05 * sf)      # 50 ms

    n_times = len(times)
    win_starts = np.arange(first_sig_idx, n_times - win_size + 1, step_size)

    # Average PAC over time window and plot topomap
    info = epochs.info  # or create an Info object manually

    n_windows = len(win_starts)
    ncols = 4
    nrows = int(np.ceil(n_windows / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows))
    axes = axes.flatten()  # flatten in case n_windows < nrows*ncols

    for i, start in enumerate(win_starts):
        end = start + win_size
        
        # Average PAC over subjects and time window
        avg_data = g1_ave[:, start:end, :].mean(axis=1).mean(axis=0)  # (n_channels,)
        
        # Highlight significant channels in this window
        chan_sig_mask = sig_mask[start:end, :].any(axis=0)
        
        # Plot topomap on the corresponding axis
        mne.viz.plot_topomap(avg_data, info, axes=axes[i], show=False,
                            mask=chan_sig_mask,
                            mask_params=dict(marker='o', markerfacecolor='w', markeredgewidth=1.5, markersize=8))
        
        axes[i].set_title(f"{times[start]:.2f}-{times[end-1]:.2f}s", fontsize=10)

    # Hide any unused subplots
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f'Significant electrodes in {cond1} vs {cond2} comparison', fontsize=16)
    plt.tight_layout()
    plt.show()

    # Save figure
    plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_topomap_sig_electrodes.png"), dpi=300)

    plt.close('all')


Processing comparison: O_BL_plan vs Y_BL_plan
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub12\preproc\analysis\s1_pac_sub12_BL_epochs_plan-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 10 columns
102 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Reading D:\BonoKat\research project\# study 1\eeg_data\set\O\s1_pac_sub14\preproc\analysis\s1_pac_sub14_BL_epochs_plan-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -500.00 ...     500.00 ms
        0 CTF compensation matrices available
Adding metadata with 10 columns
104 matching events found
No baseline cor

Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 60
Adjacency matrix shape: (16560, 16560)
stat_fun(H1): min=1.6150173683431863e-08 max=19.231996188924875
Running initial clustering …
Found 186 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

T_obs_mean = 1.0515822170926792
cluster_p_values = [0.969 1.    1.    1.    1.    1.    1.    0.362 1.    1.    0.847 1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    0.991 0.998 1.    1.    1.    1.    0.999 1.    1.    0.993
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    0.705
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    0.999 1.
 1.    1.    1.    0.791 1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    0.998 1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    0.502 1.    1.
 1.    1.    1.    1.    1.    1.

...

T_obs_mean = 1.032281630738485
cluster_p_values = [1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    0.989 1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    0.22  1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    0.999 1.    1.    1.    1.
 1.    1.    1.    1.    0.679 1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 0.987 1.    1.    1.    1.    1.    1. 

Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 60
Adjacency matrix shape: (16560, 16560)
stat_fun(H1): min=3.457958763075576e-09 max=17.58660676107788
Running initial clustering …
Found 162 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

T_obs_mean = 1.239579501753714
cluster_p_values = [1.    1.    1.    1.    1.    1.    0.845 1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    0.899 1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    0.568 1.    1.    0.003 1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.   ]
O_

Stop here

_____________________________

In [226]:
# Highlight clusters on top of T-map

plt.figure(figsize=(19, 10))
plt.imshow(T_obs.T, aspect='auto', origin='lower', cmap='plasma', vmin=0, vmax=8)
plt.colorbar(label='T-value')

# overlay cluster outlines
for i, p_val in enumerate(cluster_p):
    if p_val < 0.05:
        mask = clusters[i]
        plt.contour(mask.T, levels=[0.5], colors='white', linewidths=1.5)

plt.yticks(
    ticks=np.arange(n_channels),
    labels=ch_names
)

plt.xticks(
    ticks=np.linspace(0, len(epochs.times) - 1, 8),   # 8 evenly spaced ticks
    labels=np.round(np.linspace(epochs.times[0], epochs.times[-1], 8), 2)  # seconds
)
    
plt.title(f"Significant {pac_name} clusters: {cond1} vs {cond2}")
plt.xlabel("Time (s)")
plt.ylabel("Channel")
plt.show()

# Save topomap figures and data for each condition
plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_sig_cluster_heatmap.png"), dpi=300)



In [ ]:
# Plot the cluster-averaged timecourse
alpha = 0.05
sig_clusters = [clusters[i] for i, p in enumerate(cluster_p) if p < alpha]

if len(sig_clusters) > 0:
    mask = sig_clusters[0]   # first cluster
    time_mask = mask.any(axis=1)      # which time points
    chan_mask = mask.any(axis=0)      # which channels
    true_times = epochs.times[time_mask]

    g1_mean = g1_ave.mean(axis=0)  # (times, channels)
    g2_mean = g2_ave.mean(axis=0)

    g1_cluster_ts = g1_mean[time_mask][:, chan_mask].mean(axis=1)
    g2_cluster_ts = g2_mean[time_mask][:, chan_mask].mean(axis=1)

    plt.figure(figsize=(8, 4))
    plt.plot(g1_cluster_ts, label="O group")
    plt.plot(g2_cluster_ts, label="Y group")
    plt.legend()

    plt.xticks(
    ticks=np.linspace(0, len(true_times) - 1, 8),   # 8 evenly spaced ticks
    labels=np.round(np.linspace(true_times[0], true_times[-1], 8), 2)  # seconds
    )
    plt.title(f"Average {pac_name} within significant cluster: {cond1} vs {cond2}")
    plt.xlabel("Time (s)")
    plt.ylabel("PAC")
    plt.show()

    # Save figure
    plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_ave_sig_cluster_timecourse.png"), dpi=300)


In [ ]:
# Plot topomap with significant electrodes highlighted
# Combine all significant clusters
sig_mask = np.zeros_like(clusters[0], dtype=bool)
for i, p_val in enumerate(cluster_p):
    if p_val < alpha_threshold:
        sig_mask |= clusters[i]

# Find first significant time index
first_sig_idx = np.where(sig_mask.any(axis=1))[0][0]

# Define sliding windows
sf = 1 / (times[1] - times[0])  # sampling frequency from time vector
win_size = int(0.1 * sf)        # 100 ms
step_size = int(0.05 * sf)      # 50 ms

n_times = len(times)
win_starts = np.arange(first_sig_idx, n_times - win_size + 1, step_size)

# Average PAC over time window and plot topomap
info = epochs.info  # or create an Info object manually

n_windows = len(win_starts)
ncols = 4
nrows = int(np.ceil(n_windows / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows))
axes = axes.flatten()  # flatten in case n_windows < nrows*ncols

for i, start in enumerate(win_starts):
    end = start + win_size
    
    # Average PAC over subjects and time window
    avg_data = g1_ave[:, start:end, :].mean(axis=1).mean(axis=0)  # (n_channels,)
    
    # Highlight significant channels in this window
    chan_sig_mask = sig_mask[start:end, :].any(axis=0)
    
    # Plot topomap on the corresponding axis
    mne.viz.plot_topomap(avg_data, info, axes=axes[i], show=False,
                         mask=chan_sig_mask,
                         mask_params=dict(marker='o', markerfacecolor='w', markeredgewidth=1.5, markersize=8),
                        # cmap='plasma',
                        )
    
    axes[i].set_title(f"{times[start]:.2f}-{times[end-1]:.2f}s", fontsize=10)

# Hide any unused subplots
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.suptitle(f'Significant electrodes in {cond1} vs {cond2} comparison', fontsize=16)
plt.tight_layout()
plt.show()

# Save figure
# plt.savefig(os.path.join(figs_dir, f"{cond1}_vs_{cond2}_{pac_name}_topomap_sig_electrodes.png"), dpi=300)



______________

In [135]:
g1 = np.transpose(np.stack([r['tgPAC'] for r in group1_results]), axes=(0, 3, 2, 1))
g2 = np.transpose(np.stack([r['tgPAC'] for r in group2_results]), axes=(0, 3, 2, 1))
g1.shape, g2.shape

((24, 276, 45, 60), (23, 276, 45, 60))

In [136]:
g1_ave = np.mean(g1, axis=(2)) # (24, 60) subs x electrodes
g2_ave = np.mean(g2, axis=(2)) # (24, 60) subs x electrodes
print(g1_ave.shape, g2_ave.shape)

(24, 276, 60) (23, 276, 60)


In [137]:
print(g1.mean(), g2.mean())
print(g1.std(), g2.std())
ch_num = 10
plt.hist(g1_ave[:,:,ch_num].flatten(), bins=50, alpha=0.5, label='Group 1: O')
plt.hist(g2_ave[:,:,ch_num].flatten(), bins=50, alpha=0.5, label='Group 2: Y')

0.011166901353927079 0.010978492199354723
0.011242792899872718 0.01104597487304576


(array([ 12.,  62., 192., 338., 497., 617., 659., 634., 585., 501., 408.,
        335., 276., 205., 164., 158., 122.,  95.,  70.,  48.,  53.,  45.,
         42.,  53.,  28.,  20.,  16.,  16.,  10.,   7.,   5.,  11.,   5.,
          6.,  10.,   6.,   9.,   6.,   3.,   5.,   3.,   1.,   1.,   1.,
          0.,   2.,   1.,   0.,   3.,   2.]),
 array([0.00132897, 0.00243605, 0.00354313, 0.00465022, 0.0057573 ,
        0.00686438, 0.00797146, 0.00907854, 0.01018562, 0.01129271,
        0.01239979, 0.01350687, 0.01461395, 0.01572103, 0.01682811,
        0.0179352 , 0.01904228, 0.02014936, 0.02125644, 0.02236352,
        0.0234706 , 0.02457769, 0.02568477, 0.02679185, 0.02789893,
        0.02900601, 0.03011309, 0.03122018, 0.03232726, 0.03343434,
        0.03454142, 0.0356485 , 0.03675558, 0.03786266, 0.03896975,
        0.04007683, 0.04118391, 0.04229099, 0.04339807, 0.04450515,
        0.04561224, 0.04671932, 0.0478264 , 0.04893348, 0.05004056,
        0.05114764, 0.05225473, 0.05336181, 0.

In [ ]:
X = [g1_ave, g2_ave]  # group 1 and group 2 subject-level PAC

# Create channel adjacency matrix
sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, "eeg")

n_times = X[0].shape[1]    # 276
# n_freqs = X[0].shape[2]    # 45
n_chans = X[0].shape[2]    # 60

adjacency = combine_adjacency(n_times, sensor_adjacency)
print(f'Adjacency matrix shape: {adjacency.shape}')

degrees_of_freedom = g1.shape[0] + g2.shape[0] - 2
t_thresh = scipy.stats.t.ppf(1 - 0.01 / 2, df=degrees_of_freedom)
tail = 0 # two-tailed test

# Set the number of permutations
n_permutations = 1000

T_obs, clusters, cluster_p, H0 = mne.stats.spatio_temporal_cluster_test(
    X,
    n_permutations=n_permutations, 
    threshold=t_thresh,
    tail=tail,
    adjacency=adjacency,
    out_type="mask",
    n_jobs=-1
)


Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 60
Adjacency matrix shape: (16560, 16560)
stat_fun(H1): min=8.23470447603455e-09 max=14.428078604051716
Running initial clustering …
Found 209 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

In [139]:
# SANITY CHECKS
print(f'T_obs_mean = {T_obs.mean()}')
print(f'cluster_p_values = {cluster_p}')

alpha = 0.05  # significance threshold
significant_clusters = [i for i, p in enumerate(cluster_p) if p < alpha]
print(f"{cond1} vs {cond2} found {len(significant_clusters)} significant clusters")

T_obs_mean = 1.032281630738485
cluster_p_values = [1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    0.982 1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 0.999 1.    1.    1.    1.    1.    0.999 1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    0.999 1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    0.239 1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    0.994 1.    1.    1.    1.
 1.    1.    1.    1.    0.677 1.    1.    1.    1.    1.    1.    1.
 1.    0.999 1.    1.    1.    1.    0.999 1.    0.999 1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    0.999 1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 0.98  1.    1.    1.    1.    1.    1. 

Surrogates stats - SLOW!

In [ ]:
# SLOW WORKING CODE FOR SURROGATES (55 min / channel)

comparisons = [
    # --- Baseline (BL) ---
    ("O_BL_plan", "Y_BL_plan")]

theta = [4, 8]
alpha = [8, 12]
gamma = (30, 80, 5, 1)

for cond1, cond2 in comparisons:
    print('Processing comparison:', cond1, 'vs', cond2)
    group1, task1, stage1, block1 = conditions[cond1]
    group2, task2, stage2, block2 = conditions[cond2]

    # --- Select EEG channels only ---
    epochs, epochs_eeg1 = extract_data_for_pac(group1, task1, stage1, block1)
    _, epochs_eeg2 = extract_data_for_pac(group2, task2, stage2, block2)

    # --- Pre-allocate ---
    n_channels = len(epochs.ch_names)
    n_times = len(epochs.times)
    sf = epochs.info['sfreq']
    time = epochs.times
    info = epochs.info
    alpha_threshold = 0.05
    n_perm = 300  # number of permutations for surrogate distribution
    n_cond = 2

    # --- Loop over phase freqs and channels ---
    for phase_freq in [theta]: # or [theta, alpha]
        if phase_freq == theta:
            pac_name = 'tgPAC'
        else:
            pac_name = 'agPAC'
        
        all_sig_mean = np.zeros((n_channels, n_times))
        p = EventRelatedPac(f_pha=phase_freq, f_amp=gamma) # theta [4, 8], alpha [8, 12], gamma [30, 80]
        n_freqs = len(p.yvec)
        all_erpac = np.zeros((n_cond, n_channels, n_freqs, n_times))
        all_surrogate_diffs = np.zeros((n_channels, n_perm, n_freqs, n_times))  # 300 permutations

        for ch_idx, ch_name in enumerate(epochs.ch_names):  # for testing, use [:1] to process only first channel
            if ch_idx == 0:
                print('Processing channel:', ch_name)
                cond1_data = epochs_eeg1[:, ch_idx, :]
                cond2_data = epochs_eeg2[:, ch_idx, :]

                # ----- 1. Compute actual ERPAC values -----
                erpac1 = p.filterfit(sf, cond1_data, method="gc", n_perm=0)
                erpac2 = p.filterfit(sf, cond2_data, method="gc", n_perm=0)
                erpac_diff = erpac1 - erpac2     # shape: (n_amp, n_pha, n_times)

                # ----- 2. Build surrogate null distribution for differences -----
                combined = np.concatenate([cond1_data, cond2_data], axis=0)
                n1 = cond1_data.shape[0]
                n2 = cond2_data.shape[0]

                surrogate_diffs = np.zeros((n_perm, *erpac_diff.shape))
                # all_idx = [np.random.permutation(n1+n2) for _ in range(n_perm)]

                # Took 17min to run !
                for i in range(n_perm):
                    idx = np.random.permutation(n1+n2)
                    pseudo1 = combined[idx[:n1]]
                    pseudo2 = combined[idx[n1:]]
                    
                    erpac_pseudo1 = p.filterfit(sf, pseudo1, method="gc", n_perm=0)
                    erpac_pseudo2 = p.filterfit(sf, pseudo2, method="gc", n_perm=0)
                    
                    surrogate_diffs[i] = erpac_pseudo1 - erpac_pseudo2

                # ----- 3. Compute p-values -----
                pvals = np.mean(np.abs(surrogate_diffs) >= np.abs(erpac_diff), axis=0)

                # ----- 4. FDR correction -----
                _, pvals_fdr = fdr_correction(pvals, alpha=0.05)

                sig_mask = pvals_fdr < alpha_threshold
                sig_mean = np.where(
                np.any(sig_mask, axis=0),
                        np.nanmean(
                            np.where(
                                sig_mask, erpac_diff, np.nan), axis=0), # npmean or npmax
                        0)

                all_erpac[0, ch_idx, :, :] = np.squeeze(erpac1)
                all_erpac[1, ch_idx, :, :] = np.squeeze(erpac2)
                all_sig_mean[ch_idx, :] = sig_mean
                all_surrogate_diffs[ch_idx, :, :, :] = np.squeeze(surrogate_diffs)
                
        # Save intermediate results
        np.save(os.path.join(cond_dir, f"{cond1}_vs_{cond2}_{pac_name}_all_erpac.npy"), all_erpac)
        np.save(os.path.join(cond_dir, f"{cond1}_vs_{cond2}_{pac_name}_pvals_fdr.npy"), pvals_fdr)
        np.save(os.path.join(cond_dir, f"{cond1}_vs_{cond2}_{pac_name}_sig_mean.npy"), all_sig_mean)
        np.save(os.path.join(cond_dir, f"{cond1}_vs_{cond2}_{pac_name}_surr_diffs.npy"), all_surrogate_diffs)

In [ ]:
# Plot the difference ERPAC map with significant points
erp = erpac_diff.squeeze()       # (n_amp, n_times)
sig = sig_mask.squeeze()              # same shape

times = np.linspace(tmin, tmax, erp.shape[-1])
freqs = np.arange(30, 80+1, 5)        # e.g. array of amplitude frequencies

plt.figure(figsize=(10, 6))
plt.imshow(
    erp,
    aspect="auto",
    origin="lower",
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
)
plt.colorbar(label="ERPAC difference (Cond1 - Cond2)")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude frequency (Hz)")
plt.title("ERPAC Difference With Significant Points")

# significance overlay  
# plot mask where sig = True
sig_alpha = np.zeros_like(sig, dtype=float)
sig_alpha[sig] = 1  # transparency of sig overlay

plt.imshow(
    sig_alpha,
    aspect="auto",
    origin="lower",
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    cmap="Greys",
    alpha=sig_alpha,
)
plt.show()

**Simulated data**

In [40]:
import numpy as np
from scipy.signal import hilbert

def generate_pac_data(
    n_epochs=60,
    n_times=1000,
    sf=500,
    theta_freq=6,
    gamma_freq=80,
    pac_strength=0.3,       # coupling strength (0–1)
    noise_level=0.5,         # amplitude of gaussian noise
    random_state=None
):
    """
    Generate synthetic phase-amplitude modulated data suitable for ERPAC.
    
    Returns:
        data: ndarray (n_epochs, n_times)
    """
    rng = np.random.default_rng(random_state)
    t = np.arange(n_times) / sf
    
    data = np.zeros((n_epochs, n_times))

    for ep in range(n_epochs):

        # base theta oscillation
        theta = np.sin(2 * np.pi * theta_freq * t +
                       rng.uniform(0, 2*np.pi))
        
        # gamma oscillation with PAC
        gamma = np.sin(2 * np.pi * gamma_freq * t +
                       rng.uniform(0, 2*np.pi))
        
        # amplitude modulation: gamma amplitude depends on theta phase
        amp_mod = 1 + pac_strength * np.sin(2 * np.pi * theta_freq * t)
        pac_signal = amp_mod * gamma
        
        # combine and add noise
        data[ep] = pac_signal + 0.5 * theta + noise_level * rng.normal(size=n_times)

    return data


In [41]:
sf = 500
n_epochs = 60
n_times = 1000

# Strong PAC
cond1_data = generate_pac_data(
    n_epochs=n_epochs,
    n_times=n_times,
    sf=sf,
    pac_strength=0.5,      # <- stronger coupling
    random_state=1
)

# Weak PAC
cond2_data = generate_pac_data(
    n_epochs=n_epochs,
    n_times=n_times,
    sf=sf,
    pac_strength=0.1,      # <- weaker coupling
    random_state=2
)


In [42]:
print(cond1_data.shape, cond2_data.shape)


(60, 1000) (60, 1000)


In [91]:
erpac1.squeeze().shape

(45, 1000)

In [58]:
import numpy as np
from tensorpac import EventRelatedPac

erp = EventRelatedPac(f_pha=[4, 8], f_amp=(30, 80, 5, 1))

# ----- 1. Compute actual ERPAC values -----
erpac1 = erp.filterfit(sf, cond1_data, method="gc", n_perm=0)
erpac2 = erp.filterfit(sf, cond2_data, method="gc", n_perm=0)
erpac_diff = erpac1 - erpac2     # shape: (n_amp, n_pha, n_times)

# ----- 2. Build surrogate null distribution for differences -----
combined = np.concatenate([cond1_data, cond2_data], axis=0)
n1 = cond1_data.shape[0]
n2 = cond2_data.shape[0]
n_perm = 1000

surrogate_diffs = np.zeros((n_perm, *erpac_diff.shape))

# for i in range(n_perm):
#     idx = np.random.permutation(n1+n2)
#     pseudo1 = combined[idx[:n1]]
#     pseudo2 = combined[idx[n1:]]
    
#     erpac_pseudo1 = erp.filterfit(sf, pseudo1, method="gc", n_perm=0)
#     erpac_pseudo2 = erp.filterfit(sf, pseudo2, method="gc", n_perm=0)
    
#     surrogate_diffs[i] = erpac_pseudo1 - erpac_pseudo2

# # ----- 3. Compute p-values -----
# pvals = np.mean(np.abs(surrogate_diffs) >= np.abs(erpac_diff), axis=0)

# # ----- 4. FDR correction -----
# from mne.stats import fdr_correction
# _, pvals_fdr = fdr_correction(pvals, alpha=0.05)

# sig_mask = pvals_fdr < 0.05


Event Related PAC object defined
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC


In [59]:
cond1_data.shape

(60, 1000)

In [60]:
erpac1.shape

(45, 1, 1000)

In [61]:
combined.shape

(120, 1000)

In [50]:
surrogate_diffs.shape

(1000, 1, 1, 1000)

In [63]:
combined.shape

(120, 1000)

In [ ]:
# Took 17min to run !
for i in range(n_perm):
    idx = np.random.permutation(n1+n2)
    pseudo1 = combined[idx[:n1]]
    pseudo2 = combined[idx[n1:]]
    
    erpac_pseudo1 = erp.filterfit(sf, pseudo1, method="gc", n_perm=0)
    erpac_pseudo2 = erp.filterfit(sf, pseudo2, method="gc", n_perm=0)
    
    surrogate_diffs[i] = erpac_pseudo1 - erpac_pseudo2

# ----- 3. Compute p-values -----
pvals = np.mean(np.abs(surrogate_diffs) >= np.abs(erpac_diff), axis=0)

# ----- 4. FDR correction -----
from mne.stats import fdr_correction
_, pvals_fdr = fdr_correction(pvals, alpha=0.05)

sig_mask = pvals_fdr < 0.05

    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extract phases (n_pha=1) and amplitudes (n_amps=45)
    Compute Gaussian-Copula ERPAC
    Extrac

In [ ]:
np.save('surrogates_diffs_test', surrogate_diffs)
np.save('erpac_diff_test', erpac_diff)

# surrogate_diffs = np.load('surrogates_diffs_test.npy')
# erpac_diff = np.load('erpac_diff_test.npy')
# ----- 3. Compute p-values -----
# pvals = np.mean(np.abs(surrogate_diffs) >= np.abs(erpac_diff), axis=0)

# # ----- 4. FDR correction -----
# from mne.stats import fdr_correction
# _, pvals_fdr = fdr_correction(pvals, alpha=0.05)

# sig_mask = pvals_fdr < 0.05

In [112]:
t = 200  # example time point

surr = surrogate_diffs.squeeze()[:, 0, t]  # shape: (1000,)
real = erpac_diff.squeeze()[0, t]

plt.hist(surr, bins=30, alpha=0.7)
plt.axvline(real, color='red')
plt.title(f"ERPAC at time index {t}")
plt.xlabel("PAC difference")


Text(0.5, 0, 'PAC difference')

In [119]:
surr.shape

(1000, 45, 1000)

In [128]:
import numpy as np
import matplotlib.pyplot as plt

# squeeze to remove any singleton dims
surr = surrogate_diffs.squeeze()    # shape (n_perm, 45, 1000)
real = erpac_diff.squeeze()         # shape (45, 1000)
mask = sig_mask.squeeze()           # shape (45, 1000)

n_perm, n_freqs, n_times = surr.shape

# --- 1. Extract freq–time indices where significant ---
sig_coords = np.argwhere(mask)   # array of (freq_idx, time_idx)

if len(sig_coords) == 0:
    raise ValueError("No significant PAC points found.")

# --- 2. Select the first 20 ---
sig_coords_20 = sig_coords[151:171]

# --- 3. Plot 20 histograms ---
fig, axes = plt.subplots(4, 5, figsize=(18, 12))
axes = axes.ravel()

for ax, (f, t) in zip(axes, sig_coords_20):

    surrogate_values = surr[:, f, t]
    real_value = real[f, t]

    ax.hist(surrogate_values, bins=30, alpha=0.7)
    ax.axvline(real_value, color="red", linewidth=2)

    ax.set_title(f"Freq {f}, Time {t}")
    ax.set_xlabel("PAC difference")
    ax.set_ylabel("Count")

plt.tight_layout()
plt.show()


In [108]:
plt.hist(surrogate_diffs.squeeze()[:, :, 0])

(array([[  0.,   0.,   5.,  66., 380., 452.,  90.,   7.,   0.,   0.],
        [  0.,   0.,   0.,  28., 394., 522.,  56.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 300., 696.,   4.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   5., 370., 605.,  20.,   0.,   0.,   0.],
        [  0.,   0.,   0.,  40., 391., 487.,  80.,   2.,   0.,   0.],
        [  0.,   1.,  17.,  92., 345., 386., 132.,  27.,   0.,   0.],
        [  0.,   3.,  20., 118., 329., 335., 161.,  30.,   4.,   0.],
        [  0.,   5.,  26., 116., 308., 348., 156.,  36.,   4.,   1.],
        [  0.,   0.,   6.,  78., 369., 416., 113.,  17.,   1.,   0.],
        [  0.,   0.,   8., 100., 341., 414., 117.,  19.,   1.,   0.],
        [  0.,   0.,   4.,  81., 369., 438.,  95.,  13.,   0.,   0.],
        [  0.,   0.,   1.,  54., 405., 450.,  82.,   8.,   0.,   0.],
        [  0.,   0.,   4.,  66., 361., 476.,  85.,   8.,   0.,   0.],
        [  0.,   0.,   0.,  29., 380., 541.,  46.,   4.,   0.,   0.],
        [  0.,   0.,

In [109]:
plt.hist(erpac_diff.squeeze()[:, 0])

(array([1., 4., 4., 3., 3., 7., 5., 8., 8., 2.]),
 array([-0.05651431, -0.0480211 , -0.0395279 , -0.03103469, -0.02254148,
        -0.01404827, -0.00555507,  0.00293814,  0.01143135,  0.01992456,
         0.02841776]),
 <BarContainer object of 10 artists>)

In [93]:
surrogate_diffs.shape

(1000, 45, 1, 1000)

In [94]:
erpac_diff.shape

(45, 1, 1000)

In [67]:
sig_mask.size

45000

In [96]:
compare = np.abs(surrogate_diffs) >= np.abs(erpac_diff)
compare.shape

(1000, 45, 1, 1000)

In [116]:
sig_mask.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [85]:
import matplotlib.pyplot as plt
import numpy as np

erp = erpac_diff.squeeze()       # (n_amp, n_times)
sig = sig_mask.squeeze()              # same shape

times = np.linspace(tmin, tmax, erp.shape[-1])
freqs = np.arange(30, 80+1, 5)        # e.g. array of amplitude frequencies

plt.figure(figsize=(10, 6))
plt.imshow(
    erp,
    aspect="auto",
    origin="lower",
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
)
plt.colorbar(label="ERPAC difference (Cond1 - Cond2)")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude frequency (Hz)")
plt.title("ERPAC Difference With Significant Points")

# significance overlay  
# plot mask where sig = True
sig_alpha = np.zeros_like(sig, dtype=float)
sig_alpha[sig] = 1  # transparency of sig overlay

plt.imshow(
    sig_alpha,
    aspect="auto",
    origin="lower",
    extent=[times[0], times[-1], freqs[0], freqs[-1]],
    cmap="Greys",
    alpha=sig_alpha,
)
plt.show()
